# Decision Tree on HAPT
In this notebook, we will try using decision tree to perform classification on the Smartphone-Based Recognition of Human Activities and Postural Transitions Data Set.

## Setup

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [3]:
X_train = np.loadtxt("../input/smartphonebased-recognition-of-hapt/Train/X_train.txt")
y_train = np.loadtxt("../input/smartphonebased-recognition-of-hapt/Train/y_train.txt").astype(int)
X_test = np.loadtxt("../input/smartphonebased-recognition-of-hapt/Test/X_test.txt")
y_test = np.loadtxt("../input/smartphonebased-recognition-of-hapt/Test/y_test.txt").astype(int)
features = np.loadtxt("../input/smartphonebased-recognition-of-hapt/features.txt", dtype=str)

In [4]:
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")

In [5]:
def map_class(a):
    dictionary = {
        1: "1 WALKING",           
        2: "2 WALKING_UPSTAIRS",  
        3: "3 WALKING_DOWNSTAIRS",
        4: "4 SITTING",           
        5: "5 STANDING",          
        6: "6 LAYING",            
        7: "7 STAND_TO_SIT",      
        8: "8 SIT_TO_STAND",      
        9: "9 SIT_TO_LIE",       
        10: "10 LIE_TO_SIT",        
        11: "11 STAND_TO_LIE",     
        12: "12 LIE_TO_STAND" 
    }
    return dictionary[a]

#y_train = np.array(list(map(map_class, y_train)))
#y_test = np.array(list(map(map_class, y_test)))

print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_test: {y_test.shape}")

## Baseline Performance

Let us see how well Decision Tree performs on the unmodified dataset.

In [36]:
clf = DecisionTreeClassifier()

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(f"Accuracy of DT on dataset: {accuracy_score(y_test, y_pred)} \n")
print(classification_report(y_test, y_pred))

The baseline accuracy is 0.81 and the baseline weighted average F1 score is 0.81. We can see it performs particularly well on class 6 (LAYING) with an F1 score of 0.99.

Now let us see the top 10 features.

In [7]:
indices = np.flip(np.argsort(clf.feature_importances_))[:10]
for i in indices:
    print(features[i], clf.feature_importances_[i])

The most important feature is tGravityAcc-Max-1, which is the maximum gravity acceleration in the X-axis within each window.

## Tree Pruning
Now we will explore tuning the maximum depth of the decision tree to see if it reduces overfitting.

In [8]:
max_depth = []
acc_gini = []
acc_entropy = []
for i in range(1, 30):
    # Gini
    clf = DecisionTreeClassifier(criterion='gini', max_depth=i)
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
    acc_gini.append(accuracy_score(y_test, pred))
    # Entropy
    clf = DecisionTreeClassifier(criterion='entropy', max_depth=i)
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
    acc_entropy.append(accuracy_score(y_test, pred))
    ####
    max_depth.append(i)
d = pd.DataFrame({'acc_gini': pd.Series(acc_gini), 'acc_entropy': pd.Series(acc_entropy), 'max_depth': pd.Series(max_depth)})
# Visualize accuracy by max depth
plt.figure(figsize=(8, 6))
plt.plot('max_depth','acc_gini', data=d, label='gini')
plt.plot('max_depth','acc_entropy', data=d, label='entropy')
plt.xlabel('max_depth')
plt.ylabel('accuracy')
plt.legend()
plt.savefig('depth.png', bbox_inches = "tight")

As we can see, accuracy is similar from around a depth of around 8 onwards.

Credit: https://towardsdatascience.com/decision-tree-build-prune-and-visualize-it-using-python-12ceee9af752

In [35]:
clf = DecisionTreeClassifier(max_depth=10)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(f"Accuracy of DT(max_depth=10) on dataset: {accuracy_score(y_test, y_pred)} \n")
print(classification_report(y_test, y_pred))

The accuracy and weighted average F1 score with a maximum depth of 10 for the decision tree is very similar to the baseline, which has no maximum depth. This suggests that a decision tree depth is 10 is sufficient for this classification task, even though there are 561 features.

## Imbalanced Data

Let us check if the data is balanced.

In [9]:
values, counts = np.unique(y_train, return_counts=True)

plt.figure(figsize=(8, 6))
fig = sns.barplot(x=values,y=counts)
fig.set(xlabel='Activities', ylabel='Num. of instances', title="Training Instances by Activity")
fig.set_xticklabels(["1 WALKING", "2 WALKING_UPSTAIRS", "3 WALKING_DOWNSTAIRS", "4 SITTING", "5 STANDING", "6 LAYING",
                     "7 STAND_TO_SIT", "8 SIT_TO_STAND", "9 SIT_TO_LIE", "10 LIE_TO_SIT", "11 STAND_TO_LIE", "12 LIE_TO_STAND"]
                    , rotation=90)
plt.savefig('y_train.png', bbox_inches = "tight")

In [10]:
values, counts = np.unique(y_test, return_counts=True)

plt.figure(figsize=(8, 6))
fig = sns.barplot(x=values,y=counts)
fig.set(xlabel='Activities', ylabel='Num. of instances', title="Test Instances by Activity")
fig.set_xticklabels(["1 WALKING", "2 WALKING_UPSTAIRS", "3 WALKING_DOWNSTAIRS", "4 SITTING", "5 STANDING", "6 LAYING",
                     "7 STAND_TO_SIT", "8 SIT_TO_STAND", "9 SIT_TO_LIE", "10 LIE_TO_SIT", "11 STAND_TO_LIE", "12 LIE_TO_STAND"]
                    , rotation=90)
plt.savefig('y_test.png', bbox_inches = "tight")

We can see that both the training data and the test data are heavily imbalanced. The majority classes are 1-6 and the minority classes are 7-12. This means that evaluation metrics could become misleading to interpret and the model might overfit to majority classses.

### Rebalancing with SMOTE

To mitigate this problem, we will use SMOTE to balance the data.

In [11]:
from imblearn.over_sampling import SMOTE

In [12]:
smote = SMOTE(random_state=42)
X_train_smoted, y_train_smoted = smote.fit_resample(X_train, y_train)
X_test_smoted, y_test_smoted = smote.fit_resample(X_test, y_test)

In [13]:
values, counts = np.unique(y_train_smoted, return_counts=True)
sns.barplot(x=values,y=counts)

 After SMOTE, all classes have the same number of instances as the one with the most originally.

In [14]:
clf.fit(X_train_smoted, y_train_smoted)

y_pred = clf.predict(X_test)

print(f"Accuracy of DT on SMOTEd dataset: {accuracy_score(y_test, y_pred)} \n")
print(classification_report(y_test, y_pred))

Using SMOTE on the training data, we obtain a very small improvement in performance. Both the accuracy and weighted average F1 score increased to 0.83, an increase of about 0.02 or 2%.

## Dimensionality Reduction
Among 561 features given in the dataset, there must be some unnecessary attributes. Therefore, we try to perform dimensionality reduction with several techniques.

### LDA
We first try LDA, as it is able to use information from class labels.

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [17]:
steps = [('lda', LinearDiscriminantAnalysis(n_components=11)), ('dt', DecisionTreeClassifier())]
model = Pipeline(steps=steps)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(f"Accuracy of DT on LDA-transformed dataset: {accuracy_score(y_test, y_pred)} \n")
print(classification_report(y_test, y_pred))

After LDA, the model's performance improved significantly from the baseline. Both the accuracy and weighted average F1 score increased to 0.93, an increase of about 0.12 or 12%. The performance on the majority classes 1-6 is generally very good with F1 scores around 0.90 or above. The performance on the minority classes 7-12 is not as good, with F1 scores around between 0.5 to 0.7. 

In [19]:
accs = []
for i in range(1, len(np.unique(y_train))):
    steps = [('lda', LinearDiscriminantAnalysis(n_components=i)), ('dt', DecisionTreeClassifier())]
    model = Pipeline(steps=steps)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy of DT on LDA-transformed dataset with {} components: {}".format(i, accuracy))
    accs.append(accuracy)
    
plt.figure(figsize=(8, 6))
fig = sns.lineplot(x=np.arange(1, 12),y=accs)
fig.set(xlabel='Num of components', ylabel='Accuracy', title="Accuracy of LDA by num of components")
plt.savefig('lda.png', bbox_inches = "tight")

The accuracy is very similar for 7-11 components, at around 0.93. Therefore we try LDA with the least number of components that is able to achieve high performance, 7.

In [20]:
steps = [('lda', LinearDiscriminantAnalysis(n_components=7)), ('dt', clf)]
model = Pipeline(steps=steps)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(f"Accuracy of DT on LDA-transformed dataset: {accuracy_score(y_test, y_pred)} \n")
print(classification_report(y_test, y_pred))

However, the performance of the model on LDA with 7 components is not as good as with 11 components. Although the accuracy and weighted average F1 score at 0.93 is similar to that for LDA with 11 components, the macro average F1 score is lower than that for LDA with 11 components. Furthermore, there are classes with F1 score lower than 0.5, which is not the case for LDA with 11 components.

### SMOTE + LDA
Let us try to combine SMOTE and LDA to see whether performance increases.

In [21]:
X_train_smoted, y_train_smoted = smote.fit_resample(X_train, y_train)

In [22]:
steps = [('lda', LinearDiscriminantAnalysis(n_components=11)), ('dt', DecisionTreeClassifier())]
model = Pipeline(steps)

In [23]:
model.fit(X_train_smoted, y_train_smoted)

In [24]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

In [25]:
print("Accuracy of DT on SMOTEd and LDA-transformed dataset: {}".format(accuracy))
print(classification_report(y_test, y_pred))

Surprisingly, a combination of SMOTE and LDA performs more poorly than LDA alone. This could be because by training the model on a balanced dataset, it does not perform as well on the majority classes 1-6 as before. Therefore, it does not perform as well on the imbalanced test dataset.

### RFE
We will now try doing recursive feature elimination instead.

In [26]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import RFE, RFECV

In [27]:
rfe = RFE(estimator=DecisionTreeClassifier(), step=25, n_features_to_select=20)
steps = [('rfe', rfe), ('dt', DecisionTreeClassifier())]
model = Pipeline(steps=steps)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(f"Accuracy of DT on RFE dataset: {accuracy_score(y_test, y_pred)} \n")
print(classification_report(y_test, y_pred))

With an accuracy and weighted average F1 score of 0.84, RFE does not perform as as LDA. This could be because LDA is able to make use of information of class labels and is able to retain more information during dimensionality reduction compared to simply removing features.

## Conclusion
Overall, decision tree with LDA performs best on this dataset using the weighted average F1 score evaluation metric. Accuracy and weighted average F1 score are both 0.93.